In [1]:
# Importing libraries 
from binance.client import Client
import configparser
import pandas as pd

# Loading keys from config file
config = configparser.ConfigParser()
config.read_file(open('secret.cfg'))
actual_api_key = config.get('BINANCE', 'ACTUAL_API_KEY')
actual_secret_key = config.get('BINANCE', 'ACTUAL_SECRET_KEY')

client = Client(actual_api_key, actual_secret_key)

# Getting earliest timestamp availble (on Binance)
earliest_timestamp = client._get_earliest_valid_timestamp('ETHUSDT', '1d')  
# Here "ETHUSDT" is a trading pair and "1d" is time interval
print(earliest_timestamp)

# Getting historical data (candle data or kline)
candle = client.get_historical_klines("ETHUSDT", "1d", earliest_timestamp, limit=1000)


1502928000000


In [2]:
eth_df = pd.DataFrame(candle, columns=['dateTime', 'open', 'high', 'low', 'close', 'volume', 'closeTime', 'quoteAssetVolume', 'numberOfTrades', 'takerBuyBaseVol', 'takerBuyQuoteVol', 'ignore'])
eth_df.dateTime = pd.to_datetime(eth_df.dateTime, unit='ms')
eth_df.closeTime = pd.to_datetime(eth_df.closeTime, unit='ms')
eth_df.set_index('dateTime', inplace=True)
eth_df.to_csv('eth_candle.csv')

print(eth_df.tail(10))

                     open           high            low          close  \
dateTime                                                                 
2022-02-12  2927.54000000  2985.98000000  2858.30000000  2918.88000000   
2022-02-13  2918.58000000  2954.40000000  2836.70000000  2871.13000000   
2022-02-14  2871.14000000  2964.97000000  2829.01000000  2929.75000000   
2022-02-15  2929.75000000  3196.19000000  2912.06000000  3183.52000000   
2022-02-16  3183.53000000  3186.77000000  3044.03000000  3122.30000000   
2022-02-17  3122.31000000  3160.00000000  2851.55000000  2891.87000000   
2022-02-18  2891.88000000  2943.97000000  2750.00000000  2779.48000000   
2022-02-19  2779.30000000  2830.96000000  2694.45000000  2763.63000000   
2022-02-20  2763.63000000  2764.64000000  2576.00000000  2621.99000000   
2022-02-21  2621.99000000  2756.60000000  2605.11000000  2740.17000000   

                     volume               closeTime     quoteAssetVolume  \
dateTime                           

In [12]:
# Importing libraries 
from binance import ThreadedWebsocketManager
from twisted.internet import reactor 


client = Client(actual_api_key, actual_secret_key)


def streaming_data_process(msg):
    """
    Function to process the received messages
    param msg: input message
    """
    print(f"message type: {msg['e']}")
    print(f"close price: {msg['c']}")
    print(f"best ask price: {msg['a']}")
    print(f"best bid price: {msg['b']}")
    print("---------------------------")
    
# Starting the WebSocket
twm = ThreadedWebsocketManager(api_key=actual_api_key, api_secret=actual_secret_key)
twm.start()
# conn_key = bm.start_symbol_ticker_socket(symbol='ETHUSDT', callback=streaming_data_process)
twm.start_symbol_ticker_socket(callback=streaming_data_process, symbol='ETHUSDT')

'ethusdt@ticker'

message type: 24hrTicker
close price: 2738.95000000
best ask price: 2738.95000000
best bid price: 2738.94000000
---------------------------
message type: 24hrTicker
close price: 2738.94000000
best ask price: 2738.95000000
best bid price: 2738.94000000
---------------------------
message type: 24hrTicker
close price: 2739.03000000
best ask price: 2739.03000000
best bid price: 2739.02000000
---------------------------
message type: 24hrTicker
close price: 2739.33000000
best ask price: 2739.34000000
best bid price: 2739.33000000
---------------------------
message type: 24hrTicker
close price: 2739.52000000
best ask price: 2739.53000000
best bid price: 2739.52000000
---------------------------
message type: 24hrTicker
close price: 2739.53000000
best ask price: 2739.53000000
best bid price: 2739.52000000
---------------------------
message type: 24hrTicker
close price: 2738.96000000
best ask price: 2738.96000000
best bid price: 2738.95000000
---------------------------
message type: 24hrTi

In [ ]:
# stop websocket


# Stopping websocket
bm.stop_socket(conn_key)

# Websockets utilise a reactor loop from the Twisted library. Using the stop method above 
# will stop the websocket connection but it won’t stop the reactor loop so your code may 
# not exit when you expect.

# When you need to exit
reactor.stop()